# Time Series: Building to ARIMA Modeling

## Objectives

- Build simple models for time series data
- Explain auto-regressive and moving-average models
- Build time-series models with `statsmodels`


In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.model_selection import train_test_split

## Set Up

Our data will be monthly passenger data for airlines.

In [ ]:
# Read in our data
df = pd.read_csv('data/airline_passengers.csv')

df.head()

### Now What?

We should do a few things to explore (plot) and prep our data. Any ideas?

- 


In [ ]:
# Your code here!

In [ ]:
# More room for exploratory or preparatory code

In [ ]:
# And more room for code!

In [ ]:
# May want to check the index frequency, don't want freq=None

In [ ]:
# Make sure to decompose

In [ ]:
# And check for stationarity!

### Train Test Split

We're modeling, so we'll want to reserve some data as a test set.

But, there are special considerations for time-dependent data! Namely: we NEVER want to use the future in our training data - we need to be sure we're only ever learning from the past!

When developing a time series model, **we can't split our data randomly**, since the whole point is to make use of values that are near in time to other values. Typically we'll train on the earlier data points and test on the later ones.

Instead, we'll use **fixed partitioning** to designate sections for training and testing (with maybe a hold out set as well)

![](images/train-valid-test.png)



In addition to not shuffling our data when splitting, this will look different than our past uses of `train_test_split` because we don't really have an X and y in this case. Instead, we're using the past as our X, to predict the future.

One more note: when truly forecasting for the future, we'll want to retrain our models (using the parameters found during our modeling process) on ALL of the data available, since we'll want the end of the test set (last available data) when predicting out into the future.

> Bonus: If we want to cross-validate our time series, sklearn has that built out: https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-of-time-series-data
>
> This implements something called **roll-forwarding** (as opposed to fixed partitioning), which incrementally includes more data as the train and then evaluating on the next portion as test.
>
> Potentially useful additional resource on this: https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4

What we need to ask ourselves now is: how much do we want to save as test data?

In [ ]:
# We can manually train test split


In [ ]:
# Or we can use sklearn's train_test_split with shuffle=False


In [ ]:
# Let's visualize our train and test sets
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(train, label='train')
ax.plot(test, label='test')
ax.set_title('Train-Test Split');
plt.legend();

### Set Up an Evaluate Function

Bonus for y'all! Here's a quick function to explore a few metrics on our time series predictions.

Note! We're back to regression!

In [ ]:
from sklearn import metrics

def report_metrics(y_true, y_pred):
    print("Explained Variance:\n\t", metrics.explained_variance_score(y_true, y_pred))
    print("MAE:\n\t", metrics.mean_absolute_error(y_true, y_pred))
    print("RMSE:\n\t", metrics.mean_squared_error(y_true, y_pred, squared=False))
    print("r^2:\n\t", metrics.r2_score(y_true, y_pred))

You'll also often see time series models evaluated using Information Criterion, namely AIC or BIC. These are relative ways to explore which models comparatively perform better. A better model will have a lower AIC.

## Initial Modeling Approaches - Modelless Baselines!

#### What's the most naive way to make a prediction here? 

(searching for a very dumb approach - think our modelless baselines back in phase 2!)

- 


In [ ]:
# Let's implement it! Make our predictions for our test set

In [ ]:
# Then evaluate

#### Can we think of another naive way to approach these predictions?

(hint: `shift`)

- 


In [ ]:
# Implement another method!

In [ ]:
# Then evaluate

## The ARIMA Time Series Model

One of the most common methods used in time series forecasting is known as the ARIMA model, which stands for **AutoRegressive Integrated Moving Average**. ARIMA is a model that can be fitted to time series data in order to better understand or predict future points in the series.

Let's have a quick introduction to ARIMA. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:


A note - we can build individual AR models, or MA models, or just ARMA models. 

In practice, our raw time series data will not be stationary (like, ever), and the general strategy will be to take _differences_ to try to get a stationary series.

If the first-order differences of consecutive terms *themselves* do not form a stationary series, then the typical move is to consider *second-order* differences, i.e. differences of the differences.

It would be nice to have a modeling algorithm that would do our differencing for us. That is the point of **ARIMA** models. The 'I' stands for **integrated**, and the `statsmodels` tool will let us specify, in addition to the number of AR and MA terms, also the degree of difference on our raw data series that we want to use.

### Number of AR (Auto-Regressive) terms (`p`): 

`p` is the **auto-regressive** part of the model. It allows us to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to rain tomorrow if it has been raining for past 3 days. AR terms are just *lags* of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).

### Number of Differences (`d`):

`d` is the **Integrated** component of an ARIMA model. This value is concerned with the amount of differencing as it identifies the number of lag values to subtract from the current observation. Intuitively, this would be similar to stating that it is likely to rain tomorrow if the difference in amount of rain in the last *n* days is small. 

### Number of MA (Moving Average) terms (`q`): 

`q` is the **moving average** part of the model which is used to set the error of the model as a linear combination of the error values observed at previous time points in the past. MA terms form lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where `e(i)` is the difference between the moving average at ith instant and actual value.

These three distinct integer values, (p, d, q), are used to parametrize ARIMA models. Because of that, ARIMA models are denoted with the notation `ARIMA(p, d, q)`. 

A detailed article on these parameters is available [HERE](https://www.quantstart.com/articles/Autoregressive-Integrated-Moving-Average-ARIMA-p-d-q-Models-for-Time-Series-Analysis).

The seasonal ARIMA method can appear daunting because of the multiple tuning parameters involved. There are several manual ways to tune our ARIMA models, and later we'll explore how we can grid search these parameters as well!

#### Let's test it out!

[Documentation](https://www.statsmodels.org/devel/generated/statsmodels.tsa.arima.model.ARIMA.html)

In [ ]:
# Import ARIMA


In [ ]:
# From our data exploration earlier, what do we want to use as params?
# Order will be (p,d,q)
arima_order = None # should be a tuple, like (0,0,0)

In [ ]:
# Now let's fit our ARIMA model


In [ ]:
# Since it's statsmodels, we have a summary we can explore


In [ ]:
# We can also make predictions using .forecast
# Steps will be the length of our test data


In [ ]:
# How can we visualize our predictions?


In [ ]:
# Evaluate

## Random Walk Model

We can compare to a model with only a single differencing term, which is called a random walk model.

In [ ]:
# Create our random walk model
# Order will be (p,d,q)
rw_order = None # should be a tuple, like (0,0,0)

# Then fit


In [ ]:
# Check out the summary

In [ ]:
# Get predictions out

In [ ]:
# Evaluate

## One More ARIMA Model

Let's try another ARIMA model, with different terms than before

In [ ]:
# Create our next ARIMA model
# Order will be (p,d,q)
arima_order_2 = None # should be a tuple, like (0,0,0)

# Then fit


In [ ]:
# Check out the summary

In [ ]:
# Get predictions out

In [ ]:
# Evaluate

#### Discuss:

- 


In [ ]:
# Can we visualize all of our predictions?


## SARIMA or SARIMAX

Sometimes we want to have separate AR and MA terms so that we can model time series that have a **seasonal** component. That's what [**SARIMA** modeling](https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/) is all about.

In other cases we want to make use also of other predictors, or **exogenous** variables. For those situations we can use build **SARIMAX** models, which would be **S**easonal **A**uto-**R**egressive **I**ntegrated **M**oving-**A**verage models with e**X**ogenous variables.

-----

# LEVEL UP

## ACF and PACF

We have been able to reduce our AIC by chance, adding fairly random p, d, and q terms.

But we have tools to help guide us in these decisions: the autocorrelation and partial autocorrelation functions.

### PACF

In general, a partial correlation is a **conditional correlation**. It is the  amount of correlation between a variable and a lag of itself that is not explained by correlations at all lower-order-lags. If $Y_t$ is correlated with $Y_{t-1}$, and $Y_{t-1}$ is equally correlated with $Y_{t-2}$, then we should also expect to find correlation between $Y_t$ and $Y_{t-2}$. Thus, the correlation at lag 1 "propagates" to lag 2 and presumably to higher-order lags. The partial autocorrelation at lag 2 is therefore the difference between the actual correlation at lag 2 and the expected correlation due to the propagation of correlation at lag 1.

In [ ]:
plot_pacf(train.diff().dropna());

The shaded area of the graph is the confidence interval. When the correlation drops into the shaded area, that means there is no longer statistically significant correlation between lags.

For an AR process, we run a linear regression on lags according to the order of the AR process. The coefficients calculated factor in the influence of the other variables.   

Since the PACF shows the direct effect of previous lags, it helps us choose AR terms.  If there is a significant positive value at a lag, consider adding an AR term according to the number that you see.

Some rules of thumb: 

    - A sharp drop after lag "k" suggests an AR-k model.
    - A gradual decline suggests an MA.

### ACF

The autocorrelation plot of our time series is simply a version of the correlation plots we used in linear regression.  Our features this time are prior points in the time series, or the **lags**. 

We can calculate a specific covariance ($\gamma_k$) with:

${\displaystyle \gamma_k = \frac 1 n \sum\limits_{t=1}^{n-k} (y_t - \bar{y_t})(y_{t+k}-\bar{y_{t+k}})}$

In [ ]:
df = pd.DataFrame(train)
df.columns = ['lag_0']
df['lag_1'] = train.shift()
df.head()

In [ ]:
gamma_1 = sum(((df['lag_0'][1:]-df['lag_0'][1:].mean()) *\
               (df['lag_1'].dropna()-df['lag_1'].dropna().mean())))/(len(df['lag_1'])-1)
gamma_1

We then compute the Pearson correlation:

$\large\rho = \frac {\operatorname E[(y_1−\mu_1)(y_2−\mu_2)]} {\sigma_{1}\sigma_{2}} = \frac {\operatorname {Cov} (y_1,y_2)} {\sigma_{1}\sigma_{2}}$,

${\displaystyle \rho_k = \frac {\sum\limits_{t=1}^{n-k} (y_t - \bar{y})(y_{t+k}-\bar{y})} {\sum\limits_{t=1}^{n} (y_t - \bar{y})^2}}$

In [ ]:
rho = gamma_1/(df.lag_0[1:].std(ddof=0)*df.lag_1.std(ddof=0))
rho

In [ ]:
df = pd.DataFrame(train)
df.columns = ['lag_0']
df['lag_1'] = train.shift()
df['lag_2'] = train.shift(2)
df['lag_3'] = train.shift(3)
df['lag_4'] = train.shift(4)
df['lag_5'] = train.shift(5)
df.corr()

In [ ]:
list(df.corr()['lag_0'].index)
plt.bar(list(df.corr()['lag_0'].index), list(df.corr()['lag_0']));

In [ ]:
# Original data

plot_acf(train);

The above autocorrelation shows that there is correlation between lags up to about 12 weeks back.  

When Looking at the ACF graph for the original data, we see a strong persistent correlation with higher order lags. This is evidence that we should take a **first difference** of the data to remove this autocorrelation.

This makes sense, since we are trying to capture the effect of recent lags in our ARMA models, and with high correlation between distant lags, our models will not come close to the true process.

Generally, we use an ACF to predict MA terms.
Moving Average models are using the error terms of the predictions to calculate the next value.  This means that the algorithm does not incorporate the direct effect of the previous value. It models what are sometimes called **impulses** or **shocks** whose effect accounts for the propogation of correlation from one lag to the other. 

In [ ]:
plot_acf(train.diff().dropna());

This autocorrelation plot can now be used to get an idea of a potential MA term.  Our differenced series shows negative significant correlation at a lag of 1, which suggests adding 1 MA term. There is also a statistically significant 2nd term, so adding another MA is another possibility.


> If the ACF of the differenced series displays a sharp cutoff and/or the lag-1 autocorrelation is negative--i.e., if the series appears slightly "overdifferenced"--then consider adding an MA term to the model. The lag at which the ACF cuts off is the indicated number of MA terms. [Duke](https://people.duke.edu/~rnau/411arim3.htm#signatures)

Rule of thumb:
    
  - If the autocorrelation shows negative correlation at the first lag, try adding MA terms.
    
    

![alt text](images/armaguidelines.png)

The plots above suggest that we should try a 1st order differenced MA(1) or MA(2) model on our weekly gun offense data.

This aligns with our AIC scores from above.

The ACF can be used to identify the possible structure of time series data. That can be tricky going forward as there often isn’t a single clear-cut interpretation of a sample autocorrelation function.

Let's plot our training predictions, using an ARIMA model with order (1, 1, 2).

In [ ]:
aa_model = ARIMA(train, order=(1, 1, 2)).fit()
y_hat_train = aa_model.predict(typ='levels')

fig, ax = plt.subplots()
ax.plot(y_hat_train)
ax.plot(train);

In [ ]:
# Let's zoom in:

fig, ax = plt.subplots()
ax.plot(y_hat_train[50:70])
ax.plot(train[50:70]);

In [ ]:
aa_model.summary()